In [9]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO

anos_finais = range(2025, 2005, -1)
lista_geral_dfs = []

print(f"inicio: {len(anos_finais)}")

# --- Loop Principal ---
for ano in anos_finais:
    epoca_fim = ano
    epoca_inicio = ano - 1
    epoca_str = f"{epoca_inicio}-{epoca_fim}"
    
    url = f"https://fbref.com/en/comps/32/{epoca_str}/{epoca_str}-Primeira-Liga-Stats"
    
    print(f"\népoca: {epoca_str}")

    try:
        # extração com Selenium
        service = ChromeService(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.get(url)
        time.sleep(6)
        html_content = driver.page_source
        driver.quit()
        
        tables = pd.read_html(StringIO(html_content))
        dataframes_processados = {}

        def limpar_colunas(df):
            df.columns = ["_".join([str(c).strip() for c in col if c]) if isinstance(col, tuple) else str(col).strip() for col in df.columns.values]
            return df

        def selecionar_colunas_existentes(df, colunas_desejadas):
            colunas_existentes = [col for col in colunas_desejadas if col in df.columns]
            return df[colunas_existentes]
        
        def processar_tabela_secundaria(tabela, renames_dict):
            df = limpar_colunas(tabela.copy())
            nome_coluna_equipa_original = df.columns[0]
            df = df.rename(columns={nome_coluna_equipa_original: 'Equipa'})
            df_selecionado = selecionar_colunas_existentes(df, list(renames_dict.keys()))
            df_final = df_selecionado.rename(columns=renames_dict)
            
            #Força a coluna 'Equipa' a ser do tipo string
            if 'Equipa' in df_final.columns:
                df_final['Equipa'] = df_final['Equipa'].astype(str)
                
            return df_final

        #Processamento INDIVIDUAL e SEGURO de cada tabela
        try:
            df_overall = tables[0].copy().rename(columns={'Squad': 'Equipa'})
            
            #Força a coluna 'Equipa' a ser do tipo string
            if 'Equipa' in df_overall.columns:
                df_overall['Equipa'] = df_overall['Equipa'].astype(str)
                
            colunas_desejadas_overall = ['Equipa', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA', 'xGD']
            df_overall = selecionar_colunas_existentes(df_overall, colunas_desejadas_overall)
            df_overall = df_overall.rename(columns={'MP': 'JogosDisputados', 'W': 'Vitórias', 'D': 'Empates', 'L': 'Derrotas', 'GF': 'GolosMarcados', 'GA': 'GolosSofridos', 'GD': 'DiferençaDeGolos', 'Pts': 'Pontos', 'xG': 'GolosEsperados', 'xGA': 'GolosEsperadosSofridos', 'xGD': 'DiferençaDeGolosEsperados'})
            dataframes_processados['overall'] = df_overall
            print("Tabela 'Overall' processada.")
        except IndexError:
            print("Aviso: Tabela 'Overall' (índice 0) não encontrada.")

        renames_map = {
            'stats': (2, {'Equipa': 'Equipa', 'Performance_Gls': 'Golos', 'Performance_Ast': 'Assistências', 'Performance_G+A': 'Golos+Assistências', 'Expected_xG': 'GolosEsperados', 'Expected_xAG': 'GolosEsperadosAssistidos', 'Expected_npxG': 'GolosEsperadosSemPenáltis', 'Progression_PrgC': 'ConduçõesProgressivas', 'Progression_PrgP': 'PassesProgressivos'}, "Squad Standard Stats"),
            'goalkeeping': (4, {'Equipa': 'Equipa', 'Performance_GA': 'GolosSofridos', 'Performance_CS': 'JogosSemSofrerGolos', 'Performance_CS%': '%DeJogosSemSofrerGolos', 'Performance_Save%': '%DeDefesas', 'Performance_SoTA': 'RematesÀBalizaSofridos'}, "Squad Goalkeeping"),
            'goalkeeping_advanced': (6, {'Equipa': 'Equipa', 'Expected_PSxG': 'GolosEsperadosAposRemate', 'Expected_PSxG+/-': 'GolosEsperadosAposRemate+/-', 'Sweeper_#OPA': 'AcoesForaDaGrandeArea'}, "Squad Advanced Goalkeeping"),
            'shooting': (8, {'Equipa': 'Equipa', 'Standard_Gls': 'Golos', 'Standard_Sh': 'Remates', 'Standard_SoT': 'RematesÀBaliza', 'Standard_SoT%': '%DeRematesÀBaliza', 'Expected_xG': 'GolosEsperados', 'Expected_npxG': 'GolosEsperadosSemPenáltis'}, "Squad Shooting"),
            'passing': (10, {'Equipa': 'Equipa', 'Total_Cmp%': '%DePassesCompletos', 'Total_TotDist': 'DistanciaTotalPercorrida', 'Total_PrgDist': 'DistanciaProgressivaPercorrida', 'Unnamed: 17_level_0_Ast': 'Assistencias', 'Unnamed: 18_level_0_xAG': 'GolosEsperadosAssistidos', 'Unnamed: 21_level_0_KP': 'PassesChave', 'Unnamed: 25_level_0_PrgP': 'PassesProgressivos'}, "Squad Passing"),
            'passing_types': (12, {'Equipa': 'Equipa', 'Unnamed: 3_level_0_Att': 'TentativasDePasse', 'Pass Types_TB': 'BolasEmProfundidade', 'Pass Types_Sw': 'InversoesDeJogo'}, "Squad Pass Types"),
            'gca': (14, {'Equipa': 'Equipa', 'SCA_SCA': 'AcoesQueCriamRemates', 'SCA_SCA90': 'AcoesQueCriamRematesPor90minutos', 'GCA_GCA': 'AcoesQueCriamGolos', 'GCA_GCA90': 'AcoesQueCriamGolosPor90minutos'}, "Squad Goal and Shot Creation"),
            'defensive': (16, {'Equipa': 'Equipa', 'Tackles_Tkl': 'Desarmes', 'Unnamed: 15_level_0_Int': 'Intercecoes', 'Blocks_Blocks': 'Bloqueamentos', 'Unnamed: 16_level_0_Tkl+Int': 'Desarmes+Intercecoes', 'Unnamed: 17_level_0_Clr': 'Alivios'}, "Squad Defensive Actions"),
            'possession': (18, {'Equipa': 'Equipa', 'Unnamed: 2_level_0_Poss': 'PosseDeBola', 'Touches_Touches': 'ToquesNaBola', 'Carries_PrgC': 'ConducoesProgressivas', 'Receiving_PrgR': 'RececoesProgressivas'}, "Squad Possession"),
            'success': (20, {'Equipa': 'Equipa', 'Team Success_+/-': 'SaldoDeGolos+/-', 'Team Success_+/-90': 'SaldoDeGolos+/-Por90minutos', 'Team Success (xG)_xG+/-': 'SaldoDeGolosEsperados+/-', 'Team Success (xG)_xG+/-90': 'SaldoDeGolosEsperados+/-Por90minutos'}, "Playing Time (Team Success)"),
            'misc': (22, {'Equipa': 'Equipa', 'Aerial Duels_Won%': '%DeDuelosAéreosGanhos'}, "Miscellaneous Stats")
        }

        for nome_df, (indice, renames, nome_legivel) in renames_map.items():
            try:
                dataframes_processados[nome_df] = processar_tabela_secundaria(tables[indice], renames)
                print(f"Tabela '{nome_legivel}' processada.")
            except IndexError:
                print(f"Aviso: Tabela '{nome_legivel}' (índice {indice}) não encontrada.")
        
        if not dataframes_processados:
            raise ValueError("Nenhuma tabela foi extraída com sucesso para esta época.")

        # junção das Tabelas da epoca
        lista_dfs_epoca = list(dataframes_processados.values())
        dataset_epoca = lista_dfs_epoca[0]
        for df_temp in lista_dfs_epoca[1:]:
            dataset_epoca = pd.merge(dataset_epoca, df_temp, on='Equipa', how='left')

        dataset_epoca['Epoca'] = epoca_str
        lista_geral_dfs.append(dataset_epoca)
        print("sucesso")

    except Exception as e:
        print(f"erro na epoca: {epoca_str}. Erro: {e}. A continuar para a próxima epoca...")
        continue

# --- Finalização ---
print("\n--- Processo de extração de todas as épocas concluído ---")

if lista_geral_dfs:
    dataset_historico_completo = pd.concat(lista_geral_dfs, ignore_index=True)
    output_filename = 'dataset_completo.csv'
    dataset_historico_completo.to_csv(output_filename, index=False)
else:
    print("erro ao criar dataset_completo")

inicio: 20

época: 2024-2025
Tabela 'Overall' processada.
Tabela 'Squad Standard Stats' processada.
Tabela 'Squad Goalkeeping' processada.
Tabela 'Squad Advanced Goalkeeping' processada.
Tabela 'Squad Shooting' processada.
Tabela 'Squad Passing' processada.
Tabela 'Squad Pass Types' processada.
Tabela 'Squad Goal and Shot Creation' processada.
Tabela 'Squad Defensive Actions' processada.
Tabela 'Squad Possession' processada.
Tabela 'Playing Time (Team Success)' processada.
Tabela 'Miscellaneous Stats' processada.
sucesso

época: 2023-2024
Tabela 'Overall' processada.
Tabela 'Squad Standard Stats' processada.
Tabela 'Squad Goalkeeping' processada.
Tabela 'Squad Advanced Goalkeeping' processada.
Tabela 'Squad Shooting' processada.
Tabela 'Squad Passing' processada.
Tabela 'Squad Pass Types' processada.
Tabela 'Squad Goal and Shot Creation' processada.
Tabela 'Squad Defensive Actions' processada.
Tabela 'Squad Possession' processada.
Tabela 'Playing Time (Team Success)' processada.
Tabela

In [13]:
import numpy as np

# carrega o dataset bruto que foi gerado
df = pd.read_csv('dataset_completo.csv')

#problema 1: Corrigir Colunas Duplicadas

print("\n--- A corrigir colunas duplicadas ---")

# Identificar colunas com sufixos _x e _y
cols_x = [c for c in df.columns if c.endswith('_x')]
cols_y = [c for c in df.columns if c.endswith('_y')]

print(f"Colunas duplicadas encontradas (com sufixo _x): {cols_x}")

for col_x in cols_x:
    # Nome da coluna base (ex: 'Golos' de 'Golos_x')
    base_col = col_x[:-2]
    col_y = base_col + '_y'
    
    if col_y in df.columns:
        # unifica as duas colunas: preenche os nulos de uma com os valores da outra
        # na maioria dos casos, elas serão idênticas, mas esta é uma abordagem segura.
        df[base_col] = df[col_x].fillna(df[col_y])
        
        # Remover as colunas originais com sufixo
        df.drop(columns=[col_x, col_y], inplace=True)
        print(f"Coluna '{base_col}' unificada e duplicados removidos.")


#problema 2: Analisar Valores Nulos

# preenche os valores numéricos nulos com 0.
# isto assume que, se a estatística não existia, o seu valor era zero.
colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
df[colunas_numericas] = df[colunas_numericas].fillna(0)

mapeamento_nomes = {
    'Gil Vicente FC': 'Gil Vicente',
    'Vitória': 'Vitória de Guimarães'
    # Adicione outras variações que possa encontrar
}
df['Equipa'] = df['Equipa'].replace(mapeamento_nomes)

#guardar o dataset limpo

#remove quaisquer linhas que possam ter ficado totalmente vazias por algum erro
df.dropna(how='all', inplace=True)

# Guarda o novo ficheiro CSV, pronto para análise
df.to_csv('dataset_limpo.csv', index=False)

print("\ndataset limpo com sucesso e guardado como 'dataset_limpo.csv'")
print("\nA mostrar as primeiras 5 linhas do dataset limpo:")
print(df.head())


--- A corrigir colunas duplicadas ---
Colunas duplicadas encontradas (com sufixo _x): ['GolosSofridos_x', 'GolosEsperados_x', 'Golos_x', 'GolosEsperadosAssistidos_x', 'GolosEsperadosSemPenáltis_x', 'PassesProgressivos_x']
Coluna 'GolosSofridos' unificada e duplicados removidos.
Coluna 'GolosEsperados' unificada e duplicados removidos.
Coluna 'Golos' unificada e duplicados removidos.
Coluna 'GolosEsperadosAssistidos' unificada e duplicados removidos.
Coluna 'GolosEsperadosSemPenáltis' unificada e duplicados removidos.
Coluna 'PassesProgressivos' unificada e duplicados removidos.

dataset limpo com sucesso e guardado como 'dataset_limpo.csv'

A mostrar as primeiras 5 linhas do dataset limpo:
        Equipa  JogosDisputados  Vitórias  Empates  Derrotas  GolosMarcados  \
0  Sporting CP               34        25        7         2             88   
1      Benfica               34        25        5         4             84   
2        Porto               34        22        5         7   

In [14]:
import pandas as pd
df_limpo = pd.read_csv('dataset_limpo.csv')
print(sorted(df_limpo['Equipa'].unique()))

['AVS Futebol', 'Académica', 'Arouca', 'Aves', 'B-SAD', 'Beira-Mar', 'Belenenses', 'Benfica', 'Boavista', 'Braga', 'Casa Pia', 'Chaves', 'Estoril', 'Estrela', 'Famalicão', 'Farense', 'Feirense', 'Gil Vicente', 'Leixões', 'Marítimo', 'Moreirense', 'Nacional', 'Naval', 'Olhanense', 'Paços de Ferreira', 'Penafiel', 'Portimonense', 'Porto', 'Rio Ave', 'Santa Clara', 'Sporting CP', 'Tondela', 'Trofense', 'União', 'União de Leiria', 'Vitória Setúbal', 'Vitória de Guimarães', 'Vizela']
